CHAPTER 5

In [1]:
import pandas as pd

In [70]:
path = "data/nyc-parking-violations-2020.csv"
df = pd.read_csv(path, usecols=[
    "Plate ID", "Registration State", "Vehicle Make", "Vehicle Color", 
    "Violation Time", "Street Name"])

In [72]:
%%timeit
total = len(df.index)
without_any_nans = len(df.dropna().index)
(total - without_any_nans) * 100

1.72 s ± 11.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
%%timeit
# loss with subset features
subset = ["Plate ID", "Registration State", "Vehicle Make", "Street Name"]
(len(df) - len(df.dropna(subset=subset))) * 100

subset = ["Plate ID", "Registration State", "Street Name"]
(len(df.index) - len(df.dropna(subset=subset).index)) * 100

2.87 s ± 15.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
%%timeit
# How many rows would you eliminate if you required at least three non-null 
# values from the four columns Plate ID, Registration State, Vehicle Make, and
# Street Name

rows_with_at_least_3_non_nans = len(
    df[
        (
            df['Plate ID'].notnull().astype(int) + 
            df['Registration State'].notnull().astype(int) + 
            df['Vehicle Make'].notnull().astype(int) + 
            df['Street Name'].notnull().astype(int)
        ) >= 3
    ]
)
total - rows_with_at_least_3_non_nans

1.32 s ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%%timeit
# Which of the columns you've imported has the greatest number of NaN values
df.isnull().astype(int).sum()

1.41 s ± 15.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
%%timeit
# Null data is bad, but there is plenty of bad non-null data, too. For example,
# many cars with BLANKPLATE as a plate ID were ticketed. Turn these into NaN 
# values, and rerun the previous query.

df['Plate ID'] = df['Plate ID'].replace('BLANKPLATE', pd.NA)
df.isnull().astype(int).sum()

1.92 s ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [79]:
import polars as pl

In [80]:
path = "data/nyc-parking-violations-2020.csv"
df = pl.read_csv(path, columns=[
    "Plate ID", "Registration State", "Vehicle Make", "Vehicle Color", 
    "Violation Time", "Street Name"])

In [81]:
%%timeit
total = len(df)
without_any_nans = len(df.drop_nulls())
(total - without_any_nans) * 100

22.8 ms ± 709 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [83]:
%%timeit
# loss with subset features
subset = ["Plate ID", "Registration State", "Vehicle Make", "Street Name"]
(len(df) - len(df.drop_nulls(subset=subset))) * 100

subset = ["Plate ID", "Registration State", "Street Name"]
(len(df) - len(df.drop_nulls(subset=subset))) * 100

44 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [85]:
%%timeit
# How many rows would you eliminate if you required at least three non-null 
# values from the four columns Plate ID, Registration State, Vehicle Make, and
# Street Name

rows_with_at_least_3_non_nans = len(
    df.filter(
        (
            df['Plate ID'].is_not_null().cast(int) + 
            df['Registration State'].is_not_null().cast(int) + 
            df['Vehicle Make'].is_not_null().cast(int) + 
            df['Street Name'].is_not_null().cast(int)
        ) >= 3)
)
len(df) - rows_with_at_least_3_non_nans

192 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [86]:
%%timeit
# Which of the columns you've imported has the greatest number of NaN values
df.null_count()

846 ns ± 9.27 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [87]:
%%timeit
# Null data is bad, but there is plenty of bad non-null data, too. For example,
# many cars with BLANKPLATE as a plate ID were ticketed. Turn these into NaN 
# values, and rerun the previous query.

(
    df
    .with_columns(df['Plate ID'].replace('BLANKPLATE', None).alias('Plate ID'))
    .null_count()
)

40.6 ms ± 87.9 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


2. The goal of this exercise is to find the average age of celebrities who died February–July 2016

In [109]:
path = "data/celebrity_deaths_2016.csv"
df = pd.read_csv(path, usecols=["dateofdeath", "age"], parse_dates=["dateofdeath"])

In [110]:
# add new column with month
df["month"] = df["dateofdeath"].map(lambda x: x.month)

In [111]:
# change index to month
df.reset_index(inplace=True)
df = df.set_index(keys=['month'], drop=True).drop(columns=["index"])

In [112]:
# sort df by index
df.sort_index(inplace=True)

In [113]:
# clean all nonintegers in age column
df["age"] = df["age"].replace(r"[a-zA-Z/ ._-]*", "", regex=True).str.slice(0, 2)

df["age"] = pd.to_numeric(df["age"], errors='coerce')
df.dropna(inplace=True)

In [114]:
# age to int
df["age"] = df["age"].astype(int)

In [115]:
# find avg age from feb to july
df.loc[2:7, "age"].mean()

np.float64(75.30396873120866)

In [116]:
import datetime as dt
# mean age value in period [2016-02-15, 2016-07-15]
df.reset_index(inplace=True, drop=False)
df.set_index(keys=["dateofdeath"], drop=True, inplace=True)

start = df.index.searchsorted(dt.datetime(2016, 2, 15))
end = df.index.searchsorted(dt.datetime(2016, 7, 15))
df[start:end]['age'].mean()

np.float64(75.2947935368043)

In [ ]:
#top 5 cause of death
df = pd.read_csv(path, usecols=["dateofdeath", "age", "causeofdeath"], parse_dates=["dateofdeath"])
df["causeofdeath"].value_counts()[:5]

causeofdeath
cancer               248
heart attack         125
traffic collision     56
lung cancer           51
pneumonia             50
Name: count, dtype: int64

In [101]:
#top 5 cause of death with unknown
df["causeofdeath"].replace(pd.NA, "unknown").value_counts()[:5]

causeofdeath
unknown               5008
 cancer                248
 heart attack          125
 traffic collision      56
 lung cancer            51
Name: count, dtype: int64

In [1]:
import polars as pl

In [28]:
path = "data/celebrity_deaths_2016.csv"
df = pl.read_csv(
    path, columns=["dateofdeath", "age"], 
    try_parse_dates=True, 
    ignore_errors=True, 
    schema=pl.Schema(
        {
            "dateofdeath": pl.Date,
            "name": pl.String, 
            "age": pl.String,
            "bio": pl.String,
            "causeofdeath": pl.String
        }))

In [29]:
# add new column with month
df = df.with_columns(pl.col("dateofdeath").dt.month().alias("month"))
df

dateofdeath,age,month
date,str,i8
2016-01-01,"""71""",1
2016-01-01,"""74""",1
2016-01-01,"""79""",1
2016-01-01,"""45""",1
2016-01-01,"""83""",1
…,…,…
2016-12-27,"""74""",12
2016-12-27,"""85""",12
2016-12-27,"""83""",12


In [30]:
# sort by month
df.sort(by=["month"])

dateofdeath,age,month
date,str,i8
2016-01-01,"""71""",1
2016-01-01,"""74""",1
2016-01-01,"""79""",1
2016-01-01,"""45""",1
2016-01-01,"""83""",1
…,…,…
2016-12-27,"""74""",12
2016-12-27,"""85""",12
2016-12-27,"""83""",12


In [53]:
# clean all nonintegers in age column
df = (
        df
        .with_columns(pl.col("age").replace(r"^[0-9]*", None).str.slice(0,2)
                      .cast(int, wrap_numerical=True)).drop_nulls()
    )


In [55]:
# find avg age from feb to july
df.filter((pl.col("month")>=2) & (pl.col("month") <= 7)).select("age").mean()

age
f64
75.417978


In [ ]:
import datetime as dt
# mean age value in period [2016-02-15, 2016-07-15]
start = dt.datetime(2016, 2, 15)
end = dt.datetime(2016, 7, 15)
df.filter((pl.col("dateofdeath")>=start) & (pl.col("dateofdeath") <= end)).select("age").mean()


age
f64
75.414281


In [62]:
#top 5 cause of death
df = pl.read_csv(path, columns=["causeofdeath"])
df["causeofdeath"].drop_nulls().value_counts(sort=True)[:5]

causeofdeath,count
str,u32
""" cancer""",248
""" heart attack""",125
""" traffic collision""",56
""" lung cancer""",51
""" pneumonia""",50


In [91]:
#top 5 cause of death with unknown
df["causeofdeath"].fill_null("unknown").value_counts(sort=True)[:5]

causeofdeath,count
str,u32
"""unknown""",5008
""" cancer""",248
""" heart attack""",125
""" traffic collision""",56
""" lung cancer""",51


3. Fill in missing data from the famous Titanic data set

In [3]:
%pip install xlrd


Note: you may need to restart the kernel to use updated packages.


In [46]:
path = "data/titanic3.xls"
df = pd.read_excel(path)

# df.to_csv('data/titanic3.csv', index=False) # for arrow

In [47]:
# which columns contain null values
df.isna().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [49]:
df["age"] = df["age"].interpolate()

In [50]:
df[df["fare"].isna()]  = df[df["fare"] < 400]['fare'].mean().astype(int)

In [51]:
df = df.dropna(subset=["embarked"])

In [52]:
# df["home.dest"] = df["home.dest"].fillna(df["home.dest"].mode()[0])


In [55]:
'''
replace NaN values in the home.dest column with the most common value 
from that person's embarked column
'''
most_common_destinations = pd.Series()

for name in df['embarked'].dropna().unique():
    a = df[df['embarked']==name]['home.dest'].value_counts()
    print(a)
    most_common_destinations.loc[name] = a
        
    
most_common_destinations      

home.dest
New York, NY                            29
London                                  14
Cornwall / Akron, OH                     9
Wiltshire, England Niagara Falls, NY     8
Sweden Winnipeg, MN                      7
                                        ..
Devonport, England                       1
Tokyo, Japan                             1
North Evington, England                  1
St Ives, Cornwall / Houghton, MI         1
Antwerp, Belgium / Stanton, OH           1
Name: count, Length: 293, dtype: int64
home.dest
New York, NY                             33
Paris, France                             7
Haverford, PA / Cooperstown, NY           5
Ottawa, ON                                5
Paris / Montreal, PQ                      4
                                         ..
?Havana, Cuba                             1
St James, Long Island, NY                 1
Gallipolis, Ohio / ? Paris / New York     1
Albany, NY                                1
Austria Niagara Falls, NY   

S     home.dest
New York, NY                        ...
C     home.dest
New York, NY                        ...
Q     home.dest
Ireland Chicago, IL                 ...
31          home.dest
31    1
Name: count, dtype: int64
dtype: object

In [150]:
df[['home.dest', 'embarked']]

,home.dest,embarked
0,"St Louis, MO",S
1,"Montreal, PQ / Chesterville, ON",S
2,"Montreal, PQ / Chesterville, ON",S
3,"Montreal, PQ / Chesterville, ON",S
4,"Montreal, PQ / Chesterville, ON",S
...,...,...
1304,NaN,C
1305,NaN,C
1306,NaN,C
1307,NaN,C


In [159]:
df['home.dest'] = df['home.dest'].fillna(df['embarked'])

In [152]:
df[['home.dest', 'embarked']]

,home.dest,embarked
0,"St Louis, MO",S
1,"Montreal, PQ / Chesterville, ON",S
2,"Montreal, PQ / Chesterville, ON",S
3,"Montreal, PQ / Chesterville, ON",S
4,"Montreal, PQ / Chesterville, ON",S
...,...,...
1304,C,C
1305,C,C
1306,C,C
1307,C,C


In [160]:
df['home.dest'] = df['home.dest'].replace(most_common_destinations)

In [161]:
df[['home.dest', 'embarked']]

,home.dest,embarked
0,"St Louis, MO",S
1,"Montreal, PQ / Chesterville, ON",S
2,"Montreal, PQ / Chesterville, ON",S
3,"Montreal, PQ / Chesterville, ON",S
4,"Montreal, PQ / Chesterville, ON",S
...,...,...
1304,"New York, NY",C
1305,"New York, NY",C
1306,"New York, NY",C
1307,"New York, NY",C


In [167]:
df = None

In [1]:
%pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 6.1 MB/s eta 0:00:0000:0100:01m
Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as po

In [56]:
df = po.read_csv("data/titanic3.csv")

In [44]:
d1 = df.group_by("embarked", "home.dest").len().drop_nulls()
d2 = d1.select(po.col("embarked"), po.col("len")).group_by("embarked").max()

In [45]:
d1.join(d2, on=["len", "embarked"])

embarked,home.dest,len
str,str,u32
"""S""","""New York, NY""",29
"""Q""","""Ireland New York, NY""",4
"""Q""","""Ireland Chicago, IL""",4
"""C""","""New York, NY""",33


In [60]:
import pandas as pd
df = pd.read_csv('data/nyc-parking-violations-2020.csv', usecols=["Plate ID", "Registration State", 
"Vehicle Make", "Vehicle Color", "Street Name"])

In [61]:
# distinct vehicle colors
len(df["Vehicle Color"].drop_duplicates())

1897

In [62]:
# top 30 colors
df["Vehicle Color"].value_counts()[:30]

Vehicle Color
WH       2344858
GY       2307704
BK       2066374
WHITE    1061234
BL        775124
RD        483298
BLACK     465110
GREY      306787
BROWN     292348
SILVE     191477
GR        182929
BLUE      178298
RED       161693
TN        120576
BR        102204
YW         98700
BLK        91539
OTHER      60245
GREEN      58765
GL         54851
GRY        46527
MR         42812
GRAY       40854
WHT        35433
YELLO      32792
WHI        29760
OR         28100
BK.        27830
WT         25583
WT.        24593
Name: count, dtype: int64

In [63]:
# dictionary
colormap = {'WH': 'WHITE', 'GY':'GRAY',
'BK':'BLACK', 'BL':'BLUE',
'RD':'RED', 'GR':'GRAY',
'TN':'TAN', 'BR':'BROWN',
'YW':'YELLO', 'BLK':'BLACK',
'GRY':'GRAY', 'WHT':'WHITE',
'WHI':'WHITE', 'OR':'ORANG',
'BK.':'BLACK', 'WT':'WHITE',
'WT.':'WHITE'}


In [64]:
df["Vehicle Color"] = df["Vehicle Color"].replace(colormap)
len(df["Vehicle Color"].drop_duplicates())

1880

In [ ]:
df[:50]

In [66]:
print(len(df['Vehicle Make'].drop_duplicates()))
df['Vehicle Make'].value_counts()[:30]

5211


Vehicle Make
TOYOT    1395273
HONDA    1343265
FORD     1328063
NISSA    1119587
CHEVR     711464
FRUEH     530846
ME/BE     530473
JEEP      490977
BMW       488545
DODGE     462646
HYUND     357747
LEXUS     293752
ACURA     247954
INTER     231149
INFIN     230237
GMC       214300
SUBAR     206609
VOLKS     192747
AUDI      182256
HIN       181538
KIA       163151
CHRYS     153200
ISUZU     146872
MAZDA     141137
ROVER     102892
CADIL     102236
MITSU      94846
VOLVO      92175
NS/OT      86825
LINCO      81403
Name: count, dtype: int64

In [67]:
import re
def clean(value: str) -> str:
    if not isinstance(value, str):
        return np.nan
    return re.sub(r'[^\w\s]','', str(value).upper())

In [68]:
df['Vehicle Make'] = df['Vehicle Make'].apply(clean)

In [72]:
df["Vehicle Make"].isnull().sum()

np.int64(62420)

In [73]:
len(df["Street Name"].drop_duplicates())

57758

In [40]:
df["Street Name"].value_counts().head(50)

Street Name
Broadway                180225
3rd Ave                 133003
5th Ave                  78211
2nd Ave                  75533
Madison Ave              75419
Lexington Ave            62859
1st Ave                  58491
Queens Blvd              58423
8th Ave                  54641
WB ATLANTIC AVE @ CL     54298
WB SEAGIRT BLVD @ CR     53142
7th Ave                  51583
6th Ave                  50750
EB HORACE HARDING EX     50263
Amsterdam Ave            46657
EB CONDUIT BLVD @ GL     46185
SB MAIN ST @ 82ND DR     45274
NB SPRINGFIELD BLVD      44446
Jamaica Ave              42859
EB E 233RD ST @ KATO     42727
37th Ave                 39865
Park Ave                 36140
WB ATLANTIC AVE @ SH     34989
Roosevelt Ave            34702
WB GOETHALS RD N @ J     34210
WB LINDEN BLVD @ LIN     34187
SB FRANCIS LEWIS BLV     34026
Fulton St                33491
EB HILLSIDE AVE @ 25     33382
Columbus Ave             32971
SB WEST ST @ LEROY S     32225
White Plains Rd          31

In [74]:
import numpy as np
def clean_streets(value: str) -> str:
    if not isinstance(value, str):
        return np.nan
    return value[:value.rfind('@')].strip().upper()

In [75]:
df["Street Name"] = df["Street Name"].apply(clean_streets)

In [76]:
df["Street Name"].isnull().sum()

np.int64(1417)

In [77]:
df.dropna(subset=["Street Name"], inplace=True)

In [80]:
len(df["Registration State"].drop_duplicates())

68

In [92]:
df["Registration State"].str.isnumeric().sum()

np.int64(24530)

In [93]:
def clean_states(value: str) -> str:
    if not isinstance(value, str):
        return np.nan
    return value.strip().upper()


In [94]:
df["Registration State"] = df["Registration State"].apply(clean_states)

In [95]:
df

,Plate ID,Registration State,Vehicle Make,Street Name,Vehicle Color
0,J58JKX,NJ,HONDA,43 S,BLACK
1,KRE6058,PA,MEBE,UNION S,BLACK
2,444326R,NJ,LEXUS,CLERMONT AVENU,BLACK
3,F728330,OH,CHEVR,DIVISION AV,NaN
4,FMY9090,NY,JEEP,GRAND S,GREY
...,...,...,...,...,...
12495729,62161MM,NY,FORD,3RD AV,BROWN
12495730,GYE7330,NY,HONDA,PELHAM PARK D,BLACK
12495731,HNY4802,NY,FORD,LYDIG AV,GRAY
12495732,T687081C,NY,TOYOT,E 68 STREE,BLACK


In [96]:
df = None

In [8]:
from pyarrow import compute as pc
print(pc.case_when([{'0': True, '1': False}, {'0': False, '1': True}, {'0': False, '1': True}], [-1,3, 11], 10).to_pylist())

[-1, 10, 10]
